In [ ]:
import utility as ut
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

In [ ]:
# File paths
GANTRY_CRANE_PARAMETERS_JSON_PATH = "best_parameters.json"
SLIDING_MODE_PARAMETERS_JSON_PATH = "SMC_parameters.json"
RESULTS_PATH = "SGD_results_SMC/"

In [ ]:
def get_rise_index(time_series, setpoint):
    time_series = np.array(time_series)

    if time_series[0] > setpoint:
        time_series = -time_series
        setpoint = -setpoint

    # Cari nilai awal dari time series
    start_value = time_series[0]
    # Hitung 10% dan 90% dari nilai maksimum
    threshold_10 = start_value + 0.1 * np.abs(setpoint - start_value)
    threshold_90 = start_value + 0.9 * np.abs(setpoint - start_value)

    # Cari index ketika time series memiliki nilai terdekat dengan 10% dan 90% threshold
    rise_10_index = len(time_series) - 1
    for i in range(len(time_series)):
        if time_series[i] > threshold_10:
            rise_10_index = i
            break

    rise_90_index = len(time_series) - 1
    for i in range(len(time_series)):
        if time_series[i] > threshold_90:
            rise_90_index = i
            break

    return rise_10_index, rise_90_index

In [ ]:
def get_settling_index(time_series, setpoint, threshold=0.02):
    time_series = np.array(time_series)

    # Toleransi terhadap steady state
    bound = np.abs(setpoint - time_series[0]) * np.abs(threshold)
    if setpoint == 0:
        bound = threshold
          
    lower_bound = setpoint - bound
    upper_bound = setpoint + bound
        
    # Cari waktu ketika time series pertama kali melewati threshold 2%
    settling_index = len(time_series) - 1   # Nilai default jika tidak ditemukan
    for i in range(len(time_series) - 1, 0, -1):
        if time_series[i] < lower_bound or time_series[i] > upper_bound:
            settling_index = i
            break

    return settling_index

In [ ]:
def get_overshoot_index(time_series, setpoint):
    time_series = np.array(time_series)

    if time_series[0] > setpoint:
        time_series = -time_series
        setpoint = -setpoint

    # Cari index ketika time series pertama kali melewati setpoint
    pass_index = len(time_series)   # Nilai default jika tidak ditemukan
    for i in range(len(time_series)):
        if time_series[i] > setpoint:
            pass_index = i
            break

    # Jika time series tidak pernah melewati setpoint
    if pass_index == len(time_series):
        return pass_index
    
    # Cari index ketika time series mencapai nilai maksimum
    overshoot_index = np.argmax(time_series[pass_index:]) + pass_index

    return overshoot_index

In [ ]:
def get_RMSE_settle(settled_time_series, setpoint):
    time_series = np.array(settled_time_series)

    # Hitung RMSE terhadap setpoint
    RMSE = np.sqrt(np.mean((time_series - setpoint)**2))
    
    return RMSE

In [ ]:
DURATION = 60  # duration in seconds
DT = 0.0001  # time increment in seconds
# Create a time array
time_array = np.arange(0, DURATION + DT, DT)
NUM_STEPS = len(time_array)

SHOW_PLOTS = False

In [ ]:
# Load gantry crane parameters
GANTRY_CRANE_PARAMETERS = ut.load_json("best_parameters.json")
print(GANTRY_CRANE_PARAMETERS)

In [ ]:
from model import Simulator

simulator = Simulator(DT, NUM_STEPS)

In [ ]:
def cost_function(smc_parameters, variations):
    sum_cost = 0
    for index, variation in enumerate(variations):
        initial_conditions = {
            "x": variation["initial_condition"][0],
            "l": variation["initial_condition"][1],
            "theta": variation["initial_condition"][2],
        }

        setpoints = {
            "x": variation["setpoint"][0],
            "l": variation["setpoint"][1],
        }

        simulator.simulate_with_SMC(
            GANTRY_CRANE_PARAMETERS,
            smc_parameters,
            setpoints=setpoints,
            initial_conditions=initial_conditions,
        )
        simulation_result = simulator.get_results()

        time = simulation_result["time"]
        trolley_position = simulation_result["trolley_position"]
        cable_length = simulation_result["cable_length"]
        sway_angle = simulation_result["sway_angle"]

        # Calculate rise time, settling time, overshoot, and RMSE for x
        rise_10_index_x, rise_90_index_x = get_rise_index(trolley_position, setpoints["x"])
        rise_time_x = time[rise_90_index_x] - time[rise_10_index_x]
        settling_index_x = get_settling_index(trolley_position, setpoints["x"])
        settling_time_x = time[settling_index_x] - time[0]
        overshoot_index_x = get_overshoot_index(trolley_position, setpoints["x"])
        overshoot_x = trolley_position[overshoot_index_x] - setpoints["x"] if overshoot_index_x < NUM_STEPS else 0
        RMSE_x = get_RMSE_settle(trolley_position[settling_index_x:], setpoints["x"])

        # Calculate rise time, settling time, overshoot, and RMSE for l
        rise_10_index_l, rise_90_index_l = get_rise_index(cable_length, setpoints["l"])
        rise_time_l = time[rise_90_index_l] - time[rise_10_index_l]
        settling_index_l = get_settling_index(cable_length, setpoints["l"])
        settling_time_l = time[settling_index_l] - time[0]
        overshoot_index_l = get_overshoot_index(cable_length, setpoints["l"])
        overshoot_l = cable_length[overshoot_index_l] - setpoints["l"]  if overshoot_index_l < NUM_STEPS else 0
        RMSE_l = get_RMSE_settle(cable_length[settling_index_l:], setpoints["l"])

        # Calculate settling time and RMSE for theta
        settling_index_theta = get_settling_index(sway_angle, 0, threshold=0.01) # threshold = 0.01 rad = 0.57 deg
        settling_time_theta = time[settling_index_theta] - time[0]
        RMSE_theta = get_RMSE_settle(sway_angle[settling_index_theta:], 0)
        max_amplitude_theta = np.max(np.abs(sway_angle))

        cost = (
            rise_time_x**2 + settling_time_x**2 + overshoot_x**2 + RMSE_x**2 +
            rise_time_l**2 + settling_time_l**2 + overshoot_l**2 + RMSE_l**2 +
            settling_time_theta**2 + RMSE_theta**2 + max_amplitude_theta**2
        )

        sum_cost += cost

        if SHOW_PLOTS:
            print("Initial condition: ", initial_conditions)
            print("Setpoint: ", setpoints)
            figure, axis = plt.subplots(4, 2, figsize=(12, 6), sharex=True)
            plt.rcParams.update({"font.size": 10})
            figure.suptitle(f"Variasi {index+1} - Fungsi biaya: {round(cost, 5)}", fontsize=12)
            ut.add_to_subplot(axis[0, 0], simulation_result["time"], simulation_result["trolley_motor_pwm"], ylabel="PWM motor troli (simulasi)", color=plt.get_cmap("tab20")(0))
            ut.add_to_subplot(axis[0, 1], simulation_result["time"], simulation_result["hoist_motor_pwm"], ylabel="PWM motor angkut (simulasi)", color=plt.get_cmap("tab20")(2))
            ut.add_to_subplot(axis[1, 0], simulation_result["time"], simulation_result["trolley_motor_voltage"], ylabel="Tegangan motor troli (V) (simulasi)", color=plt.get_cmap("tab20")(4))
            ut.add_to_subplot(axis[1, 1], simulation_result["time"], simulation_result["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V) (simulasi)", color=plt.get_cmap("tab20")(6))
            ut.add_to_subplot(axis[2, 0], simulation_result["time"], simulation_result["trolley_position"], ylabel="Posisi troli (m) (simulasi)", color=plt.get_cmap("tab20")(8))
            ut.add_to_subplot(axis[2, 0], simulation_result["time"], simulation_result["trolley_speed"], ylabel="Kecepatan troli (m/s) (simulasi)", color=plt.get_cmap("tab20")(8), linestyle="--")
            ut.add_to_subplot(axis[2, 1], simulation_result["time"], simulation_result["cable_length"], ylabel="Panjang tali (m) (simulasi)", color=plt.get_cmap("tab20")(10))
            ut.add_to_subplot(axis[2, 1], simulation_result["time"], simulation_result["cable_speed"], ylabel="Kecepatan tali (m/s) (simulasi)", color=plt.get_cmap("tab20")(10), linestyle="--")
            ut.add_to_subplot(axis[3, 0], simulation_result["time"], np.rad2deg(simulation_result["sway_angle"]), "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))
            ut.add_to_subplot(axis[3, 1], simulation_result["time"], np.rad2deg(simulation_result["sway_angle"]), "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))
            plt.show()

    # mean_cost = sum_cost / len(variations)
    
    return sum_cost

In [ ]:
optimize_range = {
    "alpha1": [0.1, 10.0],
    "alpha2": [0.1, 10.0],
    "beta1": [0.1, 10.0],
    "beta2": [0.1, 10.0],
    "lambda1": [0.1, 10.0],
    "lambda2": [0.1, 10.0],
    "k1": [0.1, 10.0],
    "k2": [0.1, 10.0],
}

In [ ]:
variations_json = ut.load_json("variations.json")
print(variations_json)
variation_names = []
for variation in variations_json:
    variation_names.append(variation)
# print(variation_names)

In [ ]:
# Load SMC parameters
smc_parameters = ut.load_json(SLIDING_MODE_PARAMETERS_JSON_PATH)[
    "sliding_mode_controller"
]["parameters"]
print(smc_parameters)

NUMBER_OF_PARAMETERS = len(smc_parameters)
print("Number of parameters: ", NUMBER_OF_PARAMETERS)

In [ ]:
MAX_ITERATIONS = 100
MAX_EPISODES = 3
BATCH_SIZE = 2
LEARNING_RATE = 0.00001
GAMMA = 0.9
h = 10**-9

In [ ]:
# Choose random datasets for stochastic gradient descent
random_variation_names = random.sample(variation_names, BATCH_SIZE)
random_variations = [variations_json[index] for index in random_variation_names]
print(random_variations)
print()
best_smc_parameters = copy.deepcopy(smc_parameters)
best_cost = cost_function(best_smc_parameters, random_variations)
print(best_cost)

In [ ]:
parameters_cost_histories = {}
cost_histories = []
for episode in range(MAX_EPISODES):
    print(f"\033[92mStart episode {episode + 1} of {MAX_EPISODES} \033[0m")
    current_parameters = ut.load_json(RESULTS_PATH + "SMC_best_parameters.json")

    for parameter_ in current_parameters:
        current_parameters[parameter_]["value"] = random.uniform(
            optimize_range[parameter_][0], optimize_range[parameter_][1]
        )

    last_parameters = copy.deepcopy(current_parameters)

    parameters_momentum = copy.deepcopy(current_parameters)
    for parameter_ in parameters_momentum:
        parameters_momentum[parameter_]["value"] = 0

    diverge = False
    cost_history = np.zeros(MAX_ITERATIONS)
    for iteration in range(MAX_ITERATIONS):
        # Choose random datasets for stochastic gradient descent
        random_variation_names = random.sample(variation_names, BATCH_SIZE)
        random_variations = [variations_json[index] for index in random_variation_names]

        for index, parameter in enumerate(current_parameters):
            batch_cost = cost_function(current_parameters, random_variations)

            if batch_cost == np.inf:
                diverge = True
                parameter_value = current_parameters[parameter]["value"]
                print(
                    f"\033[91mCost is infinite at episode {episode}, iteration {iteration}, parameter: {parameter} = {parameter_value} \033[0m",
                    " " * 100,
                )
                break

            # Calculate the gradient
            current_parameters[parameter]["value"] = (
                current_parameters[parameter]["value"] + h
            )
            new_batch_cost = cost_function(current_parameters, random_variations)
            gradient = (new_batch_cost - batch_cost) / h
            current_parameters[parameter]["value"] = (
                current_parameters[parameter]["value"] - h
            )

            # Update the momentum
            temp_momentum = parameters_momentum[parameter]["value"]
            parameters_momentum[parameter]["value"] = (
                GAMMA * temp_momentum + LEARNING_RATE * gradient
            )

            value = current_parameters[parameter]["value"]
            momentum = parameters_momentum[parameter]["value"]

            print(
                f"Episode {episode + 1} of {MAX_EPISODES}, iteration {iteration + 1} of {MAX_ITERATIONS}.\033[0m",
                f"Parameter: {parameter}. Value: {value}. Cost: {batch_cost}. Gradient: {gradient}.  Momentum: {momentum}",
                " " * 100,
            )
        
        if diverge:
            print(f"\033[91mDiverge. Continue to the next episode.\033[0m", " " * 100)
            break
        
        # Update the parameter
        for parameter in current_parameters:
            temp_parameter = current_parameters[parameter]["value"]
            current_parameters[parameter]["value"] -= parameters_momentum[parameter]["value"]
            # current_parameters[parameter]["value"] -= LEARNING_RATE * gradient
            
            # Clip the parameter if it is out of the optimize range
            current_parameters[parameter]["value"] = np.clip(
                current_parameters[parameter]["value"],
                optimize_range[parameter][0],
                optimize_range[parameter][1],
            )

        # Calculate the cost
        cost = cost_function(current_parameters, random_variations)

        if cost == np.inf:
            diverge = True
            parameter_value = current_parameters[parameter]["value"]
            print(
                f"\033[91mCost is infinite at episode {episode}, iteration {iteration}.\033[0m",
                "Parameter: ",
                end="",
            )
            for parameter_ in current_parameters:
                print(parameter_, current_parameters[parameter_]["value"], end=", ")
            print()
            print("Continuing to the next episode.")
            break

        cost_history[iteration] = cost

        if cost < best_cost:
            best_cost = cost
            best_parameters = copy.deepcopy(current_parameters)
            # Save the best parameters to a JSON file
            print(f"\033[92mNew best cost found.\033[0m", "Best cost: ", best_cost, "Best parameters: ", end="")
            for parameter_ in best_parameters:
                print(parameter_, best_parameters[parameter_]["value"], end=", ")
            print()
            # Add date and time to the best_parameters
            ut.save_json(best_parameters, RESULTS_PATH + "SMC_best_parameters.json")

        if (iteration + 1) % int(MAX_ITERATIONS / 10) == 0:
            print(
                f"\033[92mCheckpoint. Episode {episode + 1} of {MAX_EPISODES}. Best cost: {best_cost} \033[0m",
                "Best parameters: ",
                end="",
            )
            for parameter_ in best_parameters:
                print(parameter_, best_parameters[parameter_]["value"], end=", ")
            print()

    cost_histories.append(cost_history)
# parameters_cost_histories[parameter] = cost_histories

In [ ]:
for episode, cost_history in enumerate(cost_histories):
    plt.plot(cost_history, label=f"Episode {episode + 1}")
    plt.title(f"Cost History of Episode {episode + 1}")
    plt.xlabel("Iteration")
    plt.ylabel("Cost")
    plt.legend()
    plt.show()

In [ ]:
# Load the best parameters
SMC_best_parameters = ut.load_json(RESULTS_PATH + "SMC_best_parameters.json")

for parameter in best_parameters:
    print(parameter, best_parameters[parameter]["value"], end=", ")
print()

# Simulate the gantry crane system using the best parameters
total_cost = 0
costs = []
for index, variation in enumerate(variations_json):
    initial_conditions = {
            "x": variation["initial_condition"][0],
            "l": variation["initial_condition"][1],
            "theta": variation["initial_condition"][2],
        }

    setpoints = {
        "x": variation["setpoint"][0],
        "l": variation["setpoint"][1],
    }

    print("Initial condition: ", initial_conditions)
    print("Setpoint: ", setpoints)
    simulator.simulate_with_SMC(
        GANTRY_CRANE_PARAMETERS,
        smc_parameters,
        setpoints=setpoints,
        initial_conditions=initial_conditions,
    )
    simulation_result = simulator.get_results()

    cost = cost_function(current_parameters, variation)
    total_cost += cost
    costs.append(cost)

    print(f"Data {index+1} - Fungsi biaya: {round(cost, 5)}")

    # Create a figure and a set of subplots
    figure, ax = plt.subplots(4, 2, figsize=(12, 6), sharex=True)
    plt.rcParams.update({"font.size": 10})
    figure.suptitle(f"Variation {index+1} - Fungsi biaya: {round(cost, 5)}", fontsize=12)
    ut.add_to_subplot(ax[0, 0], simulation_result["time"], simulation_result["trolley_motor_pwm"], ylabel="PWM motor troli (simulasi)", color=plt.get_cmap("tab20")(0))
    ut.add_to_subplot(ax[0, 1], simulation_result["time"], simulation_result["hoist_motor_pwm"], ylabel="PWM motor angkut (simulasi)", color=plt.get_cmap("tab20")(2))
    ut.add_to_subplot(ax[1, 0], simulation_result["time"], simulation_result["trolley_motor_voltage"], ylabel="Tegangan motor troli (V) (simulasi)", color=plt.get_cmap("tab20")(4))
    ut.add_to_subplot(ax[1, 1], simulation_result["time"], simulation_result["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V) (simulasi)", color=plt.get_cmap("tab20")(6))
    ut.add_to_subplot(ax[2, 0], simulation_result["time"], simulation_result["trolley_position"], ylabel="Posisi troli (m) (simulasi)", color=plt.get_cmap("tab20")(8))
    ut.add_to_subplot(ax[2, 0], simulation_result["time"], simulation_result["trolley_speed"], ylabel="Kecepatan troli (m/s) (simulasi)", color=plt.get_cmap("tab20")(8), linestyle="--")
    ut.add_to_subplot(ax[2, 1], simulation_result["time"], simulation_result["cable_length"], ylabel="Panjang tali (m) (simulasi)", color=plt.get_cmap("tab20")(10))
    ut.add_to_subplot(ax[2, 1], simulation_result["time"], simulation_result["cable_speed"], ylabel="Kecepatan tali (m/s) (simulasi)", color=plt.get_cmap("tab20")(10), linestyle="--")
    ut.add_to_subplot(ax[3, 0], simulation_result["time"], np.rad2deg(simulation_result["sway_angle"]), "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))
    ut.add_to_subplot(ax[3, 1], simulation_result["time"], np.rad2deg(simulation_result["sway_angle"]), "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))

    # ut.add_to_subplot(ax[0, 0], dataset["time"], dataset["trolley_motor_pwm"], ylabel="PWM motor troli (data)", color=plt.get_cmap("tab20")(1))
    # ut.add_to_subplot(ax[0, 1], dataset["time"], dataset["hoist_motor_pwm"], ylabel="PWM motor angkut (data)", color=plt.get_cmap("tab20")(3))
    # ut.add_to_subplot(ax[1, 0], dataset["time"], dataset["trolley_motor_voltage"], ylabel="Tegangan motor troli (V) (data)", color=plt.get_cmap("tab20")(5))
    # ut.add_to_subplot(ax[1, 1], dataset["time"], dataset["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V) (data)", color=plt.get_cmap("tab20")(7))
    # ut.add_to_subplot(ax[2, 0], dataset["time"], dataset["trolley_position"], ylabel="Posisi troli (m) (data)", color=plt.get_cmap("tab20")(9))
    # ut.add_to_subplot(ax[2, 0], dataset["time"], dataset["trolley_speed"], ylabel="Kecepatan troli (m/s) (data)", color=plt.get_cmap("tab20")(9), linestyle="--")
    # ut.add_to_subplot(ax[2, 1], dataset["time"], dataset["cable_length"], ylabel="Panjang tali (m) (data)", color=plt.get_cmap("tab20")(11))
    # ut.add_to_subplot(ax[2, 1], dataset["time"], dataset["cable_speed"], ylabel="Kecepatan tali (m/s) (data)", color=plt.get_cmap("tab20")(11), linestyle="--")
    # ut.add_to_subplot(ax[3, 0], dataset["time"], dataset["sway_angle"], "Waktu (s)", "Sudut ayun (°) (data)", color=plt.get_cmap("tab20")(13))
    # ut.add_to_subplot(ax[3, 1], dataset["time"], dataset["sway_angle"], "Waktu (s)", "Sudut ayun (°) (data)", color=plt.get_cmap("tab20")(13))
    # ax[3, 0].set_ylim(-max_sway_angle, max_sway_angle)
    # ax[3, 1].set_ylim(-max_sway_angle, max_sway_angle)
    plt.tight_layout()

    # Save the figure as SVG
    plt.savefig(RESULTS_PATH + f"figure/variation_{index+1}.svg")
    plt.show()

print("Minimum cost: ", np.min(costs))
print("Maximum cost: ", np.max(costs))
print(f"Total cost: {total_cost}")